# Verify the ProprioTunedCNN sensorprocessing

Reload a pre-trained model specified by an experiment trained by the Train-ProprioTuned-CNN notebook.

This notebook runs a number of experiments that illustrate the performance. 

In [1]:
import sys
sys.path.append("..")
from settings import Config

import pathlib
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import models, transforms
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from pprint import pprint

from behavior_cloning.demo_to_trainingdata import BCDemonstration
from sensorprocessing.sp_propriotuned_cnn import VGG19ProprioTunedRegression, VGG19ProprioTunedSensorProcessing, ResNetProprioTunedRegression, ResNetProprioTunedSensorProcessing
from robot.al5d_position_controller import RobotPosition

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
# The experiment/run we are going to run: the specified model will be created
experiment = "sensorprocessing_propriotuned_cnn"
# run = "vgg19_128"
run = "resnet50_128"
exp = Config().get_experiment(experiment, run)

Loading pointer config file: C:\Users\lboloni\.config\BerryPicker\mainsettings.yaml
Loading machine-specific config file: G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\settings-LotziYoga.yaml
No system dependent experiment file
 G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\experiment-config\LotziYoga\sensorprocessing_propriotuned_cnn\resnet50_128_sysdep.yaml,
 that is ok, proceeding.
Configuration for experiment: sensorprocessing_propriotuned_cnn/resnet50_128 successfully loaded


### Check the model for an encoding mode

Check whether we can load back the model. This loads back the model, not the sensor processing part. 

In [3]:
if exp['model'] == 'VGG19ProprioTunedRegression':
    model = VGG19ProprioTunedRegression(exp, device)
elif exp['model'] == 'ResNetProprioTunedRegression':
    model = ResNetProprioTunedRegression(exp, device)
else:
    raise Exception(f"Unknown model {exp['model']}")

if exp['loss'] == 'MSELoss':        
    criterion = nn.MSELoss()
elif exp['loss'] == 'L1Loss':
    criterion = nn.L1Loss()

modelfile = pathlib.Path(exp["data_dir"], 
                         exp["proprioception_mlp_model_file"])
print(modelfile)
assert modelfile.exists()
model.load_state_dict(torch.load(modelfile))

c:\Users\lboloni\Documents\Code\_VirtualEnvironments\Robot\Robot-venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\lboloni\Documents\Code\_VirtualEnvironments\Robot\Robot-venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


c:\Users\lboloni\Documents\Code\_TempData\BerryPicker-experiments\sensorprocessing_propriotuned_cnn\resnet50_128\proprioception_mlp.pth


C:\Users\lboloni\AppData\Local\Temp\ipykernel_46868\265961778.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(modelfile))


<All keys matched successfully>

In [4]:
#
#  FIXME: this does not work on the resnet maybe the image format is not good
#

# Run the loaded model on all the images in a specific demonstration
task = exp["proprioception_testing_task"]
demos_dir = pathlib.Path(Config()["demos"]["directory"])
task_dir = pathlib.Path(demos_dir, "demos", task)
# enc = model.to(device) # probably not need

inputlist = []
targetlist = []
with torch.no_grad():
    for demo_dir in task_dir.iterdir():
        if not demo_dir.is_dir():
            pass
        bcd = BCDemonstration(demo_dir, sensorprocessor=None)
        for i in range(bcd.trim_from, bcd.trim_to):
            sensor_readings, _ = bcd.get_image(i)
            z = model.encode(sensor_readings)
            break

print(f"The shape of the output is: {z.shape}")
print(f"One example z looks like this:\n{z}")


Cameras found: ['dev2']
There are 437 steps in this demonstration
This demonstration was recorded by the following cameras: ['dev2']
The shape of the output is: torch.Size([1, 128])
One example z looks like this:
tensor([[ 2.3152, -0.1103,  1.7146, -1.7730,  0.8909, -0.1392, -0.0121,  2.9516,
          2.5552, -1.4907,  0.0206, -1.3278,  0.0786,  2.2975, -0.6504,  3.0318,
         -0.3014,  2.7825,  0.7448, -0.1788,  1.1433,  0.6614,  0.3885,  0.7542,
          0.0675, -0.3204,  1.5375,  0.6125, -2.9732,  0.4208, -0.4143,  1.5284,
          3.4593, -3.9261,  1.3495,  0.9697,  0.2472, -0.5185, -0.3902, -0.4970,
         -0.5684,  0.1397,  1.9090, -1.9924, -0.4889, -0.2138, -1.0084, -0.3670,
         -0.1658, -0.5103, -0.0344,  0.6271,  1.3658,  0.9809,  0.5207, -3.0825,
         -1.5802,  0.7739,  0.2504, -1.2143,  0.2969,  2.5938, -1.0063,  2.9440,
          2.2842, -0.7862, -0.4598, -0.3031,  0.1334, -2.3332,  0.1624, -0.3035,
         -1.0371,  1.4727,  2.0715,  1.0237,  2.7380,  0.4

### Show how to create and use a SensorProcessing object


In [5]:
pprint(exp)
# Create the sensor processing object

# exp = Config().get_experiment("sensorprocessing_propriotuned_cnn", run)

sp = None
if exp["model"] == "VGG19ProprioTunedRegression":
    sp = VGG19ProprioTunedSensorProcessing(exp, device)
if exp["model"] == "ResNetProprioTunedRegression":
    sp = ResNetProprioTunedSensorProcessing(exp, device)
if sp is None:
    raise Exception(f"Could not instantiate sp for {exp['model']}")

{'batch_size': 32,
 'data_dir': WindowsPath('c:/Users/lboloni/Documents/Code/_TempData/BerryPicker-experiments/sensorprocessing_propriotuned_cnn/resnet50_128'),
 'epochs': 40,
 'exp_run_sys_indep_file': WindowsPath('C:/Users/lboloni/Documents/Code/_Checkouts/BerryPicker/src/experiment_configs/sensorprocessing_propriotuned_cnn/resnet50_128.yaml'),
 'freeze_feature_extractor': True,
 'group_name': 'sensorprocessing_propriotuned_cnn',
 'latent_size': 128,
 'learning_rate': 0.001,
 'loss': 'MSELoss',
 'model': 'ResNetProprioTunedRegression',
 'model_name': 'VGG-19',
 'name': 'resnet-50',
 'output_size': 6,
 'proprio_step_1': 64,
 'proprio_step_2': 16,
 'proprioception_input_file': 'train_inputs.pt',
 'proprioception_mlp_model_file': 'proprioception_mlp.pth',
 'proprioception_target_file': 'train_targets.pt',
 'proprioception_test_input_file': 'test_inputs.pt',
 'proprioception_test_target_file': 'test_targets.pt',
 'proprioception_testing_task': 'proprio_sp_validation',
 'proprioception_tr

c:\Users\lboloni\Documents\Code\_Checkouts\BerryPicker\src\sensorprocessing\..\sensorprocessing\sp_propriotuned_cnn.py:149: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sel

In [6]:
# Run the sensor processing package to process the images of a demonstration
task = exp["proprioception_testing_task"]
demos_dir = pathlib.Path(Config()["demos"]["directory"])
task_dir = pathlib.Path(demos_dir, "demos", task)
with torch.no_grad():
    for demo_dir in task_dir.iterdir():
        if not demo_dir.is_dir():
            pass
        bcd = BCDemonstration(demo_dir, sensorprocessor=None)
        for i in range(bcd.trim_from, bcd.trim_to):
            sensor_readings, _ = bcd.get_image(i)
            sensor_readings = sensor_readings.to(device)
            z = sp.process(sensor_readings)
            break

print(z)
print(z.shape)

Cameras found: ['dev2']
There are 437 steps in this demonstration
This demonstration was recorded by the following cameras: ['dev2']
sensor readings shape torch.Size([1, 3, 256, 256])
[ 2.3151774  -0.11025121  1.7145631  -1.7729801   0.8908653  -0.13919821
 -0.01207575  2.9515698   2.5551608  -1.4907382   0.02058187 -1.3277655
  0.07862917  2.2975059  -0.6503787   3.0317743  -0.30136332  2.782528
  0.7447879  -0.17877     1.1433121   0.66135246  0.38852423  0.75418967
  0.06752508 -0.32043645  1.5374613   0.6125312  -2.9732194   0.42079338
 -0.41432655  1.5284318   3.4593182  -3.9261334   1.3495319   0.96971625
  0.24715333 -0.51853585 -0.3901668  -0.49702907 -0.56835467  0.13966437
  1.908995   -1.992428   -0.48889711 -0.21382123 -1.0084422  -0.36704603
 -0.16577998 -0.5102609  -0.03442331  0.627088    1.3658031   0.98089564
  0.5206706  -3.0824513  -1.5802021   0.77388215  0.25041652 -1.2143217
  0.2968935   2.5938103  -1.0062528   2.9439943   2.2841856  -0.7861655
 -0.45979652 -0.30

In [7]:
# Run the sensor processing inside the BCDemonstration
task = exp["proprioception_testing_task"]
demos_dir = pathlib.Path(Config()["demos"]["directory"])
task_dir = pathlib.Path(demos_dir, "demos", task)
with torch.no_grad():
    for demo_dir in task_dir.iterdir():
        if not demo_dir.is_dir():
            pass
        bcd = BCDemonstration(demo_dir, sensorprocessor=sp)
        for i in range(bcd.trim_from, bcd.trim_to):
            #sensor_readings, _ = bcd.get_image(i)
            #sensor_readings = sensor_readings.to(device)
            #z = sp.process(sensor_readings)
            z = bcd.get_z(i)
            break

print(z)
print(z.shape)

Cameras found: ['dev2']
There are 437 steps in this demonstration
This demonstration was recorded by the following cameras: ['dev2']
sensor readings shape torch.Size([1, 3, 256, 256])
[ 2.3151774  -0.11025121  1.7145631  -1.7729801   0.8908653  -0.13919821
 -0.01207575  2.9515698   2.5551608  -1.4907382   0.02058187 -1.3277655
  0.07862917  2.2975059  -0.6503787   3.0317743  -0.30136332  2.782528
  0.7447879  -0.17877     1.1433121   0.66135246  0.38852423  0.75418967
  0.06752508 -0.32043645  1.5374613   0.6125312  -2.9732194   0.42079338
 -0.41432655  1.5284318   3.4593182  -3.9261334   1.3495319   0.96971625
  0.24715333 -0.51853585 -0.3901668  -0.49702907 -0.56835467  0.13966437
  1.908995   -1.992428   -0.48889711 -0.21382123 -1.0084422  -0.36704603
 -0.16577998 -0.5102609  -0.03442331  0.627088    1.3658031   0.98089564
  0.5206706  -3.0824513  -1.5802021   0.77388215  0.25041652 -1.2143217
  0.2968935   2.5938103  -1.0062528   2.9439943   2.2841856  -0.7861655
 -0.45979652 -0.30